### Load dependencies

In [4]:
import os
import shutil
import time
import pickle
import pandas as pd
import matplotlib.pyplot as pyplot
import librosa
import librosa.display
import gc
import numpy as np
import urllib

def download_rec(uri, rec_dir, filename):
    if not os.path.exists(rec_dir+'/'+uri.replace('/','_')):
        try:
            urllib.request.urlretrieve (uri, rec_dir+'/'+filename)
        except Exception as e:
            print(e)
            print('Error downloading '+uri)
        return


### Specify data paths:

1. train_set_dir - Folder where training data will be stored
2. recording_dir - Folder where audio recordings will be stored
3. sound_annotation_file - File storing template matching validation metadata
4. (Optional) sampling_rate - rate to resample training data recordings to
    

In [25]:
train_set_dir = '../train_tp/' # Folder where training data will be stored
recording_dir = '../recordings/' # Folder holding recordings

sound_annotation_files = ['../sound_annotation_files/POSP_a_tp.csv']
# File storing ROIs of detected sounds (animal calls) 
#     Required columns:
#          species
#          x1 (start time of sound)
#          x2 (end time of sound)
#          uri (recording file path)

sampling_rate = 48000 # training data recording sample rate


### Run remaining cells to generate training data

In [26]:
if not os.path.exists(recording_dir):
    os.mkdir(recording_dir)
if not os.path.exists(train_set_dir):
    os.mkdir(train_set_dir)

In [27]:
if len(sound_annotation_files)==1:
    rois= pd.read_csv(sound_annotation_files[0])
elif len(sound_annotation_files)==0:
    print('Must provide an annotation file')
elif len(sound_annotation_files)>1:
    rois = pd.read_csv(sound_annotation_files[0])
    for i in sound_annotation_files[1:]:
        tmp = pd.read_csv(sound_annotation_files[i])
        rois = pd.concat([rois,tmp])
rois.head()

,Unnamed: 0,id,recording,site,year,month,day,hour,minute,species,songtype,x1,x2,y1,y2,validated,score,site_id,url
0,0,171720534,MON_20200110_200700.p21.flac,Miraflores,2020,1,10,20,27,Porzana spiloptera,Common Song,9.705941,10.298050,1033.59375,5254.101562,present,0.361161,22307,https://arbimon.rfcx.org/api/project/urban-wet...
1,1,171720537,MON_20200110_200700.p19.flac,Miraflores,2020,1,10,20,25,Porzana spiloptera,Common Song,27.829116,28.421224,1033.59375,5254.101562,present,0.394213,22307,https://arbimon.rfcx.org/api/project/urban-wet...
2,2,171720538,MON_20200110_200700.p19.flac,Miraflores,2020,1,10,20,25,Porzana spiloptera,Common Song,46.968163,47.560272,1033.59375,5254.101562,present,0.341443,22307,https://arbimon.rfcx.org/api/project/urban-wet...
3,3,171720539,MON_20200110_200700.p19.flac,Miraflores,2020,1,10,20,25,Porzana spiloptera,Common Song,54.595918,55.188027,1033.59375,5254.101562,present,0.311693,22307,https://arbimon.rfcx.org/api/project/urban-wet...
4,4,171720542,MON_20200110_200700.p22.flac,Miraflores,2020,1,10,20,28,Porzana spiloptera,Common Song,39.172063,39.764172,1033.59375,5254.101562,present,0.360440,22307,https://arbimon.rfcx.org/api/project/urban-wet...


In [ ]:
# # For using Arbimon 2 Pattern Matching results - convert uri to full download URL

# rois['uri'] = [i[1].uri.split('detections')[0]+
#                   'site_'+str(i[1].site_id)+'/'+
#                   str(i[1].year)+'/'+
#                   str(i[1].month)+'/'+
#                   i[1].recording for i in rois.iterrows()]


In [ ]:
#rois['url'][0]

In [28]:
print('Number of ROIs for each species\n')

for i in list(set(rois.species)):
    print(str(i)+'\t\t'+str(len(rois[rois.species==i])))

Number of ROIs for each species

Porzana spiloptera		1402


In [29]:
window_length = 2 # sample time-window length in seconds
k = 0
t0 = time.time()
rec_loaded = False

for i in list(set(rois.url)): # loop over recordings
    
    k = k+1
    print(k)
        
    tmp = rois[rois.url==i]
    audio_filename = tmp.iloc[0].url.split('download/')[1]

    for c in range(len(tmp)): # loop over spectrogram ROIs
        #print(c)
        try:

            sound_start, sound_end = [tmp.iloc[c]['x1'], tmp.iloc[c]['x2']]
            species = tmp.iloc[c].species.replace(' ','_')
            
            if not os.path.exists(train_set_dir+'/'+str(species)):
                os.mkdir(train_set_dir+'/'+str(species))

            shft = ((sound_end-sound_start)-window_length)/2
            start_sample = round(sampling_rate*(sound_start+shft))
            start_sample = max(start_sample, 0)
            filename = audio_filename+'_'+str(round(start_sample/sampling_rate,2))+'-'+str(round((start_sample/sampling_rate)+window_length,2))+'.png'

            if not os.path.exists(train_set_dir+str(species)+'/'+filename):
                if not rec_loaded:
                    if not os.path.exists(recording_dir+'/'+audio_filename):
                        download_rec(tmp.iloc[0].url, recording_dir, audio_filename)
                    try:
                        audio_data, sampling_rate = librosa.load(recording_dir+audio_filename, sr=sampling_rate)
                        rec_loaded = True
                    except Exception as e:
                        print(e)
                        continue
                S = librosa.feature.melspectrogram(y = audio_data[int(start_sample): int(start_sample+round(sampling_rate*window_length))], 
                                               sr = sampling_rate,
                                               n_fft=2048, 
                                               hop_length=512, 
                                               win_length=1024)
                dpi=100
                fig = pyplot.figure(num=None, figsize=(300/dpi, 300/dpi), dpi=dpi)
                pyplot.subplot(222)
                ax = pyplot.axes()
                ax.set_axis_off()
                librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
                pyplot.savefig(train_set_dir+str(species.replace(' ','_'))+'/'+filename, bbox_inches='tight', transparent=True, pad_inches=0.0)
                pyplot.close()
                
        except Exception as e:
            #print(e)
            continue
        
    rec_loaded = False    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
